## Yahoo Fundamental

In [1]:
# def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
#     """
#     Extrat data from trading economics callendar.

#     Parameters:
#         classe_tabela (str): Class CSS from table to scrap (optional).

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')

#         if table_class:
#             tabela = soup.find('table', {'class': table_class})
#         else:
#             tabela = soup.find('table')
        
#         headers = [th.text.strip() for th in tabela.find_all('th')]
#         print(headers)
#         rows = []
#         for row in tabela.find_all('tr')[1:]:
#             cols = [td.text.strip() for td in row.find_all('td')]
#             if cols: 
#                 rows.append(cols)

#         df = pd.DataFrame(rows, columns=headers if headers else None)
        

#     except requests.exceptions.RequestException as e:
#         print(f"Error accessing URL: {e}")
#     except Exception as e:
#         print(f"Error processing data: {e}")

#     return df


In [2]:
# def get_whalewisdom(ticker: str) -> pd.DataFrame:
#     """
#     Extracts data from WhaleWisdom for a given stock ticker.

#     Parameters:
#         ticker (str): Stock ticker symbol.

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Encontrar a div com a classe "v-data-table__wrapper"
#         div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
#         if not div_wrapper:
#             raise ValueError("Div com a classe 'v-window__container' não encontrada.")

#         # Buscar a primeira tabela dentro desta div
#         tabela = div_wrapper.find('table')
#         if not tabela:
#             raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
#         # Coletar os cabeçalhos da tabela ignorando ícones
#         headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
#         # print(headers)

#         # Encontrando todas as linhas dentro da tabela (tbody > tr)
#         rows = tabela.find("tbody").find_all("tr")
#         print(rows)
#         # Extraindo os dados de cada linha
#         data = []
#         for row in rows:
#             columns = row.find_all("td")

#             # Pegando os valores correspondentes
#             institution_name = columns[0].text.strip()
#             shares_held = columns[2].text.strip()
#             portfolio_value = columns[3].text.strip()
#             percentage_ownership = columns[4].text.strip()
#             last_report_date = columns[-1].text.strip()

#             data.append({
#                 "Instituição": institution_name,
#                 "Ações Detidas": shares_held,
#                 "Valor do Portfólio": portfolio_value,
#                 "Percentual de Participação": percentage_ownership,
#                 "Última Atualização": last_report_date
#             })
#         #print(data)
    
#     except requests.exceptions.RequestException as e:
#         print(f"Erro ao acessar a URL: {e}")
#     except ValueError as e:
#         print(f"Erro na extração de dados: {e}")
#     except Exception as e:
#         print(f"Erro inesperado: {e}")
    
#     return pd.DataFrame()

In [3]:
# df_whale_s = get_whalewisdom(ticker = "ctre")
# df_whale_s

In [4]:
# base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Accept": "application/json"
# }

# response = requests.get(base_url, headers=headers)
# response.raise_for_status()

# response.text

In [ ]:
# import hashlib
# import hmac
# import base64
# import json
# import time
# import urllib.parse
# import requests

# class WhaleWisdom:
#     def __init__( secret_key, shared_key):
#         self.secret_key = secret_key
#         self.shared_key = shared_key
#         self.timestamp = self.get_timestamp()

#     def get_timestamp(self):
#         """Gera o timestamp no formato UTC"""
#         return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

#     def signature( args):
#         """Gera a assinatura HMAC-SHA1 em Base64"""
#         message = f"{args}\n{self.timestamp}".encode('utf-8')
#         hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
#         return base64.b64encode(hmac_digest).decode('utf-8')

#     def encode( string):
#         """Codifica a string para URL"""
#         return urllib.parse.quote(string, safe='')

#     def endpoint( args):
#         """Constrói a URL da API do WhaleWisdom"""
#         encoded_args = self.encode(json.dumps(args))
#         api_sig = self.signature(json.dumps(args))
#         encoded_timestamp = self.encode(self.timestamp)

#         return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

#     def request( args):
#         """Faz a requisição à API"""
#         url = self.endpoint(args)
#         response = requests.get(url)

#         if response.status_code == 200:
#             return response.text
#         else:
#             print(f"Erro {response.status_code}: {response.text}")
#             return None


# # **Exemplo de Uso**
# WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
# WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

# ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# # Obter as participações (holdings) de um gestor específico
# response = ww.request({
#     "command": "stock_lookup",
#     "symbol": "ctre"
# })

# data = json.loads(response)

# stock_id = data["stocks"][0]["id"]
# name = data["stocks"][0]["name"]
# status = data["stocks"][0]["status"]
# link = data["stocks"][0]["link"]

# stock_id

In [ ]:
# response = ww.request(
# {"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
# response

In [7]:
# response

## STOCKS



In [ ]:
# ----- Activar Ambiente -----  #
# taurus\Scripts\activate

# ----- Novo Ambiente -----  #
# python3 -m venv taurus
# pip install ipykernel yfinance pandas numpy datetime requests
# python -m ipykernel install --user --name=taurus --display-name "Python (Projeto)"

# ----- Desativar / apagar ambiente ----- #
# deactivate
# rmdir /s /q venv

SyntaxError: invalid syntax (2815492327.py, line 1)

In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib
import math
from datetime import datetime, timedelta
# from scipy.signal import argrelextrema
# import plotly.graph_objects as go
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from bs4 import BeautifulSoup
import requests
from django.core.validators import RegexValidator
from django.http import JsonResponse, Http404

# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "QUBT"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários


In [21]:
def get_sector_etf_info(sector: str, search_value: str = "info"):
        """
        Return information about symbol ETF sector.
        """
        sector_map = {
            "Technology": "XLK",
            "Financial Services": "XLF",
            "Consumer Cyclical": "XLY",
            "Healthcare": "XLV",
            "Communication Services": "XLC",
            "Industrials": "XLI",
            "Consumer Defensive": "XLP",
            "Energy": "XLE",
            "Real Estate": "XLRE",
            "Basic Materials": "XLB",
            "Utilities": "XLU",
        }

        etf_symbol = sector_map.get(sector)
        if not etf_symbol:
            return "NaN"

        ticker = yf.Ticker(etf_symbol)

        if search_value == "info":
            return ticker.info
        else:
            return ticker.info.get(search_value, "NaN")

In [28]:
from scipy.signal import argrelextrema

def get_yoy_metric(series):
        if isinstance(series, pd.Series):
            series = series.replace(0, np.nan).dropna()

            if len(series) >= 2:
                yoy_growth = ((series - series.shift(-1)) / series.shift(-1)) * 100
                yoy_growth = yoy_growth.dropna()
            else:
                yoy_growth = pd.Series("NaN", index=series.index)

        return yoy_growth

def get_cagr_metric( series):
    if isinstance(series, pd.Series):
        series = pd.to_numeric(series, errors='coerce').dropna()

        if len(series) >= 2:
            start_value = series.iloc[-1]
            end_value = series.iloc[0]
            num_years = len(series) - 1

            if start_value != 0:
                cagr = ((end_value / start_value) ** (1 / num_years)) - 1
            else:
                cagr = None
        else:
            cagr = None
    else:
        cagr = None

    cagr_percent = cagr * 100 if cagr != None else None

    return cagr_percent

def peak_detect( close, order):
    '''
    This function find the peaks of a timeframe
    '''

    if len(close) < 5:
        return [], [], 0, 0, []

    if isinstance(close, pd.Series):
        close_np = close.values  # Converte apenas os valores se for uma série Pandas
    else:
        close_np = np.array(close)

    max_idx = list(argrelextrema(close_np, comparator=np.greater, order=order)[0])
    min_idx = list(argrelextrema(close_np, comparator=np.less, order=order)[0])

    idx = max_idx + min_idx + [len(close_np)-1]
    idx.sort()

    current_idx = idx[-5:]

    start = min(current_idx)
    end = max(current_idx)

    current_pat = close_np[current_idx]

    return current_idx, current_pat, start, end, idx

def safe_round( value):
    try:
        return round(float(value), 2)
    except (ValueError, TypeError):
        return None

def convert_string_milions( value):
    if pd.isna(value) or value == "-":
        return 0

    value = str(value).strip().upper()

    try:
        if value.endswith('M'):
            return int(float(value[:-1]) * 1_000_000)
        elif value.endswith('B'):
            return int(float(value[:-1]) * 1_000_000_000)
        elif value.endswith('K'):
            return int(float(value[:-1]) * 1_000)
        else:
            return float(value.replace(",", ""))
    except Exception:
        return 0


In [29]:
yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
yahoo_symbol_balancesheet

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Ordinary Shares Number,129012000.0,77451356.0,55963334.0,29156815.0,NaN
Share Issued,129012000.0,77451356.0,55963334.0,29156815.0,NaN
Net Debt,NaN,437195.0,1880000.0,NaN,NaN
Total Debt,1181000.0,2765000.0,8266000.0,18084.0,NaN
Tangible Book Value,42742000.0,1054000.0,-5629000.0,16201744.0,NaN
...,...,...,...,...,...
Accounts Receivable,27000.0,65000.0,13000.0,0.0,NaN
Allowance For Doubtful Accounts Receivable,-3500.0,0.0,NaN,NaN,NaN
Gross Accounts Receivable,30500.0,65000.0,NaN,NaN,NaN
Cash Cash Equivalents And Short Term Investments,78945000.0,2059000.0,5308000.0,16738657.0,NaN


In [30]:
yahoo_symbol_income = yf.Ticker(symbol).income_stmt
yahoo_symbol_income

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Tax Effect Of Unusual Items,0.00,2.112000e+03,0.00,0.00,NaN
Tax Rate For Calcs,0.00,4.000000e-03,0.00,0.00,NaN
Normalized EBITDA,-21716000.00,-2.264100e+07,-26735000.00,-16895346.00,NaN
Total Unusual Items,-40532000.00,5.280000e+05,3392000.00,0.00,NaN
Total Unusual Items Excluding Goodwill,-40532000.00,5.280000e+05,3392000.00,0.00,NaN
Net Income From Continuing Operation Net Minority Interest,-68542000.00,-2.702200e+07,-25978000.00,-27898847.00,NaN
Reconciled Depreciation,3798000.00,3.307000e+06,1863000.00,8998.00,NaN
Reconciled Cost Of Revenue,261000.00,1.960000e+05,61000.00,0.00,NaN
EBITDA,-62248000.00,-2.211300e+07,-23343000.00,-16895346.00,NaN
EBIT,-66046000.00,-2.542000e+07,-25206000.00,-16904344.00,NaN


In [31]:
yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
yahoo_symbol_cashflow

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Free Cash Flow,-22249000.0,-20427000.0,-16248000.0,-6842435.0,NaN
Repurchase Of Capital Stock,-4098000.0,0.0,NaN,NaN,NaN
Repayment Of Debt,NaN,-6187034.0,0.0,-218371.0,-1609000.0
Issuance Of Debt,NaN,0.0,6960000.0,0.0,218371.0
Issuance Of Capital Stock,106766000.0,24730000.0,0.0,8606250.0,NaN
Capital Expenditure,-6036000.0,-2112000.0,-870000.0,-19391.0,NaN
Interest Paid Supplemental Data,268000.0,813000.0,0.0,0.0,NaN
Income Tax Paid Supplemental Data,NaN,0.0,0.0,0.0,0.0
End Cash Position,78945000.0,2059000.0,5308000.0,16738657.0,NaN
Beginning Cash Position,2059000.0,5308000.0,16739000.0,15196322.0,NaN


In [32]:
def get_symbol_fundamental_info(symbol: str):
    '''
    Return detailed fundamental information about asset
    '''
    try:
        yahoo_symbol_info = yf.Ticker(symbol).info
    except Exception:
        yahoo_symbol_info = {}
    try:
        yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
    except Exception:
        yahoo_symbol_balancesheet = pd.DataFrame()
    try:
        yahoo_symbol_income = yf.Ticker(symbol).income_stmt
    except Exception:
        yahoo_symbol_income = pd.DataFrame()
    try:
        yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
    except Exception:
        yahoo_symbol_cashflow = pd.DataFrame()

    # Valuation
    sector = yahoo_symbol_info.get("sector")
    
    sector_pe = get_sector_etf_info(sector, "trailingPE")

    # Dividends & BuyBacks
    eps_ann = yahoo_symbol_info.get("epsCurrentYear")
    dividend_rate = yahoo_symbol_info.get("dividendRate")

    if (
        eps_ann is not None and
        dividend_rate is not None and
        dividend_rate != 0 and
        not math.isnan(eps_ann) and
        not math.isnan(dividend_rate)
    ):
        div_coverage_rate = eps_ann / dividend_rate
    else:
        div_coverage_rate = None

    # Profitability
    # net_income
    if 'Net Income' in yahoo_symbol_income.index:
        net_income = yahoo_symbol_income.loc['Net Income']
        if pd.isna(net_income).all():
            net_income = None

    # total_revenue
    if 'Total Revenue' in yahoo_symbol_income.index:
        total_revenue = yahoo_symbol_income.loc['Total Revenue']
        if pd.isna(total_revenue).all():
            total_revenue = None

    # cost_of_revenue
    if 'Cost Of Revenue' in yahoo_symbol_income.index:
        cost_of_revenue = yahoo_symbol_income.loc['Cost Of Revenue']
        if pd.isna(cost_of_revenue).all():
            cost_of_revenue = None

    # gross_profit
    if 'Gross Profit' in yahoo_symbol_income.index:
        gross_profit = yahoo_symbol_income.loc['Gross Profit']
        if pd.isna(gross_profit).all():
            gross_profit = None

    # operating_expenses
    if 'Operating Expense' in yahoo_symbol_income.index:
        operating_expenses = yahoo_symbol_income.loc['Operating Expense']
        if pd.isna(operating_expenses).all():
            operating_expenses = None

    cagr_cost_of_revenue = get_cagr_metric(cost_of_revenue)
    if cagr_cost_of_revenue is not None and math.isnan(cagr_cost_of_revenue):
        cagr_cost_of_revenue = None
    cagr_total_revenue = get_cagr_metric(total_revenue)
    if cagr_total_revenue is not None and math.isnan(cagr_total_revenue):
        cagr_total_revenue = None
    cagr_operating_expenses = get_cagr_metric(operating_expenses)
    if cagr_operating_expenses is not None and math.isnan(cagr_operating_expenses):
        cagr_operating_expenses = None

    # Growth & NetWorth & Health
    # total_assets
    if 'Total Assets' in yahoo_symbol_balancesheet.index:
        total_assets = yahoo_symbol_balancesheet.loc['Total Assets']
        if pd.isna(total_assets).all():
            total_assets = None

    # current_liabilities
    if 'Current Liabilities' in yahoo_symbol_balancesheet.index:
        current_liabilities = yahoo_symbol_balancesheet.loc['Current Liabilities']
        if pd.isna(current_liabilities).all():
            current_liabilities = None

    # non_current_liabilities
    if 'Total Non Current Liabilities Net Minority Interest' in yahoo_symbol_balancesheet.index:
        non_current_liabilities = yahoo_symbol_balancesheet.loc['Total Non Current Liabilities Net Minority Interest']
        if pd.isna(non_current_liabilities).all():
            non_current_liabilities = None

    # total_liabilities
    if not current_liabilities.isna().all() and not non_current_liabilities.isna().all():
        total_liabilities = (current_liabilities + non_current_liabilities)
    else:
        total_liabilities = None

    # net_worth
    if not total_assets.isna().all() and not current_liabilities.isna().all() and not non_current_liabilities.isna().all():
        total_assets = total_assets.replace(0, np.nan)
        net_worth = total_assets.iloc[0] - total_liabilities.iloc[0]
    else:
        net_worth = None

    # current_assets
    if 'Current Assets' in yahoo_symbol_balancesheet.index:
        current_assets = yahoo_symbol_balancesheet.loc['Current Assets']
        if pd.isna(current_assets).all():
            current_assets = None

    # non_current_assets
    if 'Total Non Current Assets' in yahoo_symbol_balancesheet.index:
        non_current_assets = yahoo_symbol_balancesheet.loc['Total Non Current Assets']
        if pd.isna(non_current_assets).all():
            non_current_assets = None

    # short_term_debt_coverage
    if not current_assets.isna().all() and not current_liabilities.isna().all():
        short_term_debt_coverage = current_assets - current_liabilities
    else:
        short_term_debt_coverage = None

    # long_term_debt_coverage
    if not non_current_assets.isna().all() and not non_current_liabilities.isna().all():
        long_term_debt_coverage = non_current_assets - non_current_liabilities
    else:
        long_term_debt_coverage = None

    cagr_total_liabilities = get_cagr_metric(total_liabilities)
    if cagr_total_liabilities is not None and math.isnan(cagr_total_liabilities):
        cagr_total_liabilities = None

    cagr_total_assets = get_cagr_metric(total_assets)
    if cagr_total_assets is not None and math.isnan(cagr_total_assets):
        cagr_total_assets = None

    # cash_cash_equivalents
    if 'Cash And Cash Equivalents' in yahoo_symbol_balancesheet.index:
        cash_cash_equivalents = yahoo_symbol_balancesheet.loc['Cash And Cash Equivalents']
        if pd.isna(cash_cash_equivalents).all():
            cash_cash_equivalents = None

    # stockholders_equity
    if 'Stockholders Equity' in yahoo_symbol_balancesheet.index:
        stockholders_equity = yahoo_symbol_balancesheet.loc['Stockholders Equity']
        if pd.isna(stockholders_equity).all():
            stockholders_equity = None

    cagr_stockholder_equity = get_cagr_metric(stockholders_equity)
    if cagr_stockholder_equity is not None and math.isnan(cagr_stockholder_equity):
        cagr_stockholder_equity = None

    # Cashflow
    # free_cashflow
    if 'Free Cash Flow' in yahoo_symbol_cashflow.index:
        free_cashflow = yahoo_symbol_cashflow.loc['Free Cash Flow']
        if pd.isna(free_cashflow).all():
            free_cashflow = None

    # operating_cashflow
    if 'Operating Cash Flow' in yahoo_symbol_cashflow.index:
        operating_cashflow = yahoo_symbol_cashflow.loc['Operating Cash Flow']
        if pd.isna(operating_cashflow).all():
            operating_cashflow = None

    # capital_expenditure
    if 'Capital Expenditure' in yahoo_symbol_cashflow.index:
        capital_expenditure = yahoo_symbol_cashflow.loc['Capital Expenditure']
        if pd.isna(capital_expenditure).all():
            capital_expenditure = None

    # market_cap
    market_cap = yahoo_symbol_info.get('marketCap')
    if market_cap is None or (isinstance(market_cap, float) and math.isnan(market_cap)):
        market_cap = None

    # free_cashflow_yield
    if not pd.isna(market_cap) and market_cap != 0 and not free_cashflow.isna().all():
        free_cashflow_yield = ((free_cashflow.iloc[0] / market_cap) * 100)
    else:
        free_cashflow_yield = None

    # Ratios
    # current_ratio
    if not current_assets.isna().all() and not current_liabilities.isna().all():
        current_liabilities = current_liabilities.replace(0, np.nan)
        current_ratio_series = (current_assets / current_liabilities) * 100
        current_ratio_series = current_ratio_series.replace([np.inf, -np.inf], np.nan)

        if current_ratio_series.isna().all():
            current_ratio = None
        else:
            current_ratio = current_ratio_series
    else:
        current_ratio = None

    # cagr_current_ratio
    cagr_current_ratio = get_cagr_metric(current_ratio)
    if cagr_current_ratio is not None and math.isnan(cagr_current_ratio):
        cagr_current_ratio = None

    # cash_ratio
    if not cash_cash_equivalents.isna().all() and not current_liabilities.isna().all():
        current_liabilities = current_liabilities.replace(0, np.nan)
        cash_ratio_series = (cash_cash_equivalents / current_liabilities) * 100
        cash_ratio_series = cash_ratio_series.replace([np.inf, -np.inf], np.nan)

        if cash_ratio_series.isna().all():
            cash_ratio = None
        else:
            cash_ratio = cash_ratio_series
    else:
        cash_ratio = None

    # cagr_cash_ratio
    cagr_cash_ratio = get_cagr_metric(cash_ratio)
    if cagr_cash_ratio is not None and math.isnan(cagr_cash_ratio):
        cagr_cash_ratio = None

    # gross_margin
    if not gross_profit.isna().all() and not total_revenue.isna().all():
        total_revenue = total_revenue.replace(0, np.nan)
        gross_margin_series = (gross_profit / total_revenue) * 100
        gross_margin_series = gross_margin_series.replace([np.inf, -np.inf], np.nan)

        if gross_margin_series.isna().all():
            gross_margin = None
        else:
            gross_margin = gross_margin_series
    else:
        gross_margin = None

    # cagr_gross_margin
    # cagr_gross_margin = get_cagr_metric(gross_margin)

    # operating_income
    if 'Operating Income' in yahoo_symbol_income.index:
        operating_income = yahoo_symbol_income.loc['Operating Income']
        if pd.isna(operating_income).all():
            operating_income = None

    # operation_margin
    if not operating_income.isna().all() and not total_revenue.isna().all():
        total_revenue = total_revenue.replace(0, np.nan)
        operating_margin_series = (operating_income / total_revenue) * 100
        operating_margin_series = operating_margin_series.replace([np.inf, -np.inf], np.nan)

        if operating_margin_series.isna().all():
            operating_margin = None
        else:
            operating_margin = operating_margin_series
    else:
        operating_margin = None

    # cagr_operating_margin
    cagr_operating_margin = get_cagr_metric(operating_margin)
    if cagr_operating_margin is not None and math.isnan(cagr_operating_margin):
        cagr_operating_margin = None

    # profit_margin
    if not net_income.isna().all() and not total_revenue.isna().all():
        total_revenue = total_revenue.replace(0, np.nan)
        profit_margin_series = (net_income / total_revenue) * 100
        profit_margin_series = profit_margin_series.replace([np.inf, -np.inf], np.nan)

        if profit_margin_series.isna().all():
            profit_margin = None
        else:
            profit_margin = profit_margin_series
    else:
        profit_margin = None

    # cagr_profit_margin
    cagr_profit_margin = get_cagr_metric(profit_margin)
    if cagr_profit_margin is not None and math.isnan(cagr_profit_margin):
        cagr_profit_margin = None

    # return_on_equity
    if not net_income.isna().all() and not stockholders_equity.isna().all():
        stockholders_equity = stockholders_equity.replace(0, np.nan)
        return_on_equity_series = (net_income / stockholders_equity) * 100
        return_on_equity_series = return_on_equity_series.replace([np.inf, -np.inf], np.nan)

        if return_on_equity_series.isna().all():
            return_on_equity = None
        else:
            return_on_equity = return_on_equity_series
    else:
        return_on_equity = None

    # cagr_return_on_equity
    cagr_return_on_equity = get_cagr_metric(return_on_equity)
    if cagr_return_on_equity is not None and math.isnan(cagr_return_on_equity):
        cagr_return_on_equity = None

    yahoo_symbol_fundamental_info = {
        "valuation": {
            "trailingPE": yahoo_symbol_info.get("trailingPE", None),
            "sectorTrailingPE": sector_pe,
            "forwardPE": yahoo_symbol_info.get("forwardPE", None),
            "PEGRatio": yahoo_symbol_info.get("trailingPegRatio", None),
        },
        "dividends": {
            "divCoverageRate": div_coverage_rate,
            "dividendYield": yahoo_symbol_info.get("dividendYield", None),
            "fiveYearAvgDividendYield": yahoo_symbol_info.get("fiveYearAvgDividendYield", None),
        },
        "profitability": {
            "NetIncome": net_income.iloc[0],
            "TotalRevenue": total_revenue.iloc[0],
            "CostOfRevenue": cost_of_revenue.iloc[0],
            "GrossProfit": gross_profit.iloc[0],
            "OperatingExpenses": operating_expenses.iloc[0],
            "CostOfRevenueCAGR": cagr_cost_of_revenue,
            "TotalRevenueCAGR": cagr_total_revenue,
            "OperatingExpensesCAGR": cagr_operating_expenses,
        },
        "liquidity": {
            # Actual
            "TotalAssets": total_assets.iloc[0],
            "TotalLiabilities": total_liabilities.iloc[0],
            "NetWorth": net_worth,
            "CashCashEquivalents": cash_cash_equivalents.iloc[0],
            # Short Term 1y
            "ShortTermDebtCoverage": short_term_debt_coverage.iloc[0],
            "CurrentAssets": current_assets.iloc[0],
            "CurrentLiabilities": current_liabilities.iloc[0],
            # Long Term
            "LongTermDebtCoverage": long_term_debt_coverage.iloc[0],
            "NonCurrentAssets": non_current_assets.iloc[0],
            "NonCurrentLiabilities": non_current_liabilities.iloc[0],
            # Growth
            "TotalAssetsCAGR": cagr_total_assets,
            "TotalLiabilitiesCAGR": cagr_total_liabilities,
            # Stockholders Equity
            "StockholdersEquityCAGR": cagr_stockholder_equity,
            "StockholdersEquity": stockholders_equity.iloc[0],
        },
        "cashflow": {
            "FreeCashflow": free_cashflow.iloc[0],
            "OperatingCashflow": operating_cashflow.iloc[0],
            "CapitalExpenditure": capital_expenditure.iloc[0],
            "MarketCap": market_cap,
            "FreeCashflowYield": free_cashflow_yield,
        },
        "ratios": {
            # Health & Debt
            "CurrentRatio": current_ratio.iloc[0],
            "CurrentRatioCAGR": cagr_current_ratio,
            "CashRatio": cash_ratio.iloc[0],
            "CashRatioCAGR": cagr_cash_ratio,
            # Margins
            "GrossMargin": gross_margin.iloc[0],
            # "GrossMarginCAGR": cagr_gross_margin,
            "OperatingMargin": operating_margin.iloc[0],
            "OperatingMarginCAGR": cagr_operating_margin,
            "ProfitMargin": profit_margin.iloc[0],
            "ProfitMarginCAGR": cagr_profit_margin,
            "ReturnOnEquity": return_on_equity.iloc[0],
            "ReturnOnEquityCAGR": cagr_return_on_equity,
        },
        "market_risk_and_sentiment": {
            "beta": yahoo_symbol_info.get("beta", None),
            "auditRisk": yahoo_symbol_info.get("auditRisk", None),
            "boardRisk": yahoo_symbol_info.get("boardRisk", None),
            "sharesPercentSharesOut": yahoo_symbol_info.get("sharesPercentSharesOut", None),
            "recommendationMean": yahoo_symbol_info.get("recommendationMean", None),
            "targetMeanPrice": yahoo_symbol_info.get("targetMeanPrice", None)
        }
    }
    return yahoo_symbol_fundamental_info

In [33]:
yahoo_symbol_fundamental_info = get_symbol_fundamental_info(symbol=symbol)

In [38]:
yahoo_symbol_fundamental_info

{'valuation': {'trailingPE': None,
  'sectorTrailingPE': 37.58287,
  'forwardPE': -47.387096,
  'PEGRatio': None},
 'dividends': {'divCoverageRate': None,
  'dividendYield': None,
  'fiveYearAvgDividendYield': None},
 'profitability': {'NetIncome': np.float64(-68542000.0),
  'TotalRevenue': np.float64(373000.0),
  'CostOfRevenue': np.float64(261000.0),
  'GrossProfit': np.float64(112000.0),
  'OperatingExpenses': np.float64(26049000.0),
  'CostOfRevenueCAGR': None,
  'TotalRevenueCAGR': None,
  'OperatingExpensesCAGR': np.float64(15.487347932307593)},
 'liquidity': {'TotalAssets': np.float64(153559000.0),
  'TotalLiabilities': np.float64(46272000.0),
  'NetWorth': np.float64(107287000.0),
  'CashCashEquivalents': np.float64(78945000.0),
  'ShortTermDebtCoverage': np.float64(74592000.0),
  'CurrentAssets': np.float64(79151000.0),
  'CurrentLiabilities': np.float64(4559000.0),
  'LongTermDebtCoverage': np.float64(32695000.0),
  'NonCurrentAssets': np.float64(74408000.0),
  'NonCurrentLia